### User Defined Functions (UDF)

- Custom functions by Users to extend the inbuilt capabilities of Spark. 
- Define it once and use it across multiple queries.
- SQL UDF's are recommended over Python UDFs due to better optimization. Stored in Unity Catalog metastore


In [0]:
%sql

----- Defining a UDF
create or replace function gizmobox.default.get_fullname(firstname string, surname string) ----1. Define the function name and schema
returns string  --- 2. Return type of function
return concat(initcap(firstname), ' ',initcap(surname));

In [0]:
%sql
select gizmobox.default.get_fullname('john', 'smith');

In [0]:
%sql DESC FUNCTION EXTENDED gizmobox.default.get_fullname;

In [0]:
%sql
create or replace function gizmobox.default.get_payments_status(payment_status INT)
returns string
return case payment_status
when 1 then 'Success'
when 2 then 'Pending'
when 3 then 'Cancelled'
when 4 then 'Failed'
else 'Unknown'
end;

### Higher order Functions 

- Functions that lets you operate on complex data types such as arrays and maps
- They allow you to pass functions as arguments(lambda expression), apply transformations and return arrays or maps
- They are extremely useful for manipulating arrays without exploding them. 

In [0]:
%sql
create or replace temporary view order_items as 
select * from values
(1, array('smartphone', 'laptop', 'monitor')),
(2, array('tablet', 'headphones', 'smartwatch')),
(3, array('keyboard', 'mouse'))
as orders(order_id, items);


In [0]:
%sql
select * from order_items;

In [0]:
%sql

---Transforming the contents to Upper case
select order_id, transform(items, x->upper(x)) as upper_items
from order_items;

In [0]:
%sql

-- Filter items that contain only string 'smart'

select order_id, filter(items, x-> x like '%smart%') as smart_items
from order_items;

In [0]:
%sql
--- Check to see whether the order includes a value
select order_id, exists(items, x-> x = 'monitor' ) as has_monitor
from order_items;


### Arrays with more than one object

In [0]:
%sql
create or replace temp view order_items as 
select * from values
(1, array (
  named_struct('name', 'smartphone', 'price', 1000),
  named_struct('name', 'laptop', 'price', 2000),
  named_struct('name', 'monitor', 'price', 500)
)),
(2, array (
  named_struct('name', 'tablet', 'price', 300),
  named_struct('name', 'headphones' , 'price', 100),
  named_struct('name','smartwatch', 'price', 200)
)),
(3, array (
  named_struct('name', 'keyboard', 'price', 50),
  named_struct('name', 'mouse', 'price', 100)
))
as orders(order_id, items);


In [0]:
%sql
select * from order_items;

#### 1.Convert all the item names to Upper Case and Add 10% tax to each item

In [0]:
%sql
select order_id, transform(items, x -> named_struct('name', upper(x.name), 'price', round(x.price * 1.10, 2))) as items_with_tax
from order_items

#### 2. Calculate the total order amount for each of the order

In [0]:
%sql
select order_id, 
aggregate(items, 0, (acc, x) -> acc + x.price) as total_order_price
from order_items;

### MAP functions

A map is a collection of key-value pairs, like a dictionary

In [0]:
%sql
create or replace temporary view order_item_prices as
select * from values
(1, map('smartphone', 1000, 'laptop', 2000, 'monitor', 500)),
(2, map('tablet', 300, 'headphones', 100, 'smartwatch', 200)),
(3, map('keyboard', 50, 'mouse', 100))
as orders(order_id, item_prices);

In [0]:
%sql
select * from order_item_prices;

In [0]:
%sql

--- Transforming keys to upper case
select order_id, transform_keys(item_prices, (item,price) -> upper(item)) as items_upper_case
from order_item_prices;

In [0]:
%sql

-- Apply 10% tax to item prices

select order_id, transform_values(item_prices, (item,price) -> round(price * 1.10, 2)) as item_prices_with_tax
from order_item_prices

In [0]:
%sql

--- Filter prices higher than 500$

select order_id, map_filter(item_prices, (item, price) -> price > 500) as premium_items
from order_item_prices;